In [1]:
import os
import pyodbc
import pandas as pd
from IPython.display import display, Markdown
from ebmdatalab import bq

### Server connection

In [2]:
server = 'covid.ebmdatalab.net,1433'
database = 'OPENCoronaExport' 
username = 'SA'
password = 'my_secret_password' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### CHD codes
- just those in the CHD cluster, to identify cardiac events
- in order to differentiate between patients on primary and secondary prevention

In [3]:
qof_clusters = pd.read_csv('../data/QoFClusteres_CTV3Codes - Sheet1.csv')
chd_codes = qof_clusters.loc[qof_clusters['ClusterId']=='CHD','CTV3Code']
chd_codes.head()

3562    14A3.
3563    14A4.
3564    G300.
3565    G301.
3566    G3010
Name: CTV3Code, dtype: object

### CVD medicine codes
- taken from https://github.com/ebmdatalab/cvd-covid-codelist-notebook/blob/master/notebooks/cvd.codelist.ipynb

In [4]:
sql = '''
WITH bnf_codes AS (
  SELECT bnf_code FROM hscic.presentation WHERE 
    bnf_code LIKE '02%' #BNF cvd chapter 
)

SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, bnf_code, id
'''

cvd_medcodes = bq.cached_read(sql, csv_path=os.path.join('..','data','cvd_medcodes.csv'))
cvd_medcodes.head()

,type,id,bnf_code,nm
0,amp,4783111000001104,0201010AABBAAAA,Digibind 38mg powder for solution for injectio...
1,amp,20477811000001103,0201010AABCAAAB,DigiFab 40mg powder for solution for infusion ...
2,amp,3726311000001104,0201010D0AAAAAA,Digitoxin 100microgram tablets (A A H Pharmace...
3,amp,3726811000001108,0201010D0AAAAAA,Digitoxin 100microgram tablets (Focus Pharmace...
4,amp,5626411000001106,0201010D0AAAAAA,Digitoxin 100microgram tablets (Alliance Healt...


In [5]:
def codes_to_sql_where(col_name, code_list):
    where = ""
    i = 0
    for code in code_list:
        if i == 0:
            where = where + f"{col_name} = '{code}'"
        else:
            where = where + f" OR {col_name} = '{code}'"
        i+=1
    return where

In [6]:
codes_to_sql_where("CTV3Code", chd_codes.head())

"CTV3Code = '14A3.' OR CTV3Code = '14A4.' OR CTV3Code = 'G300.' OR CTV3Code = 'G301.' OR CTV3Code = 'G3010'"

In [7]:
codes_where = codes_to_sql_where("CTV3Code", chd_codes)
query = f'''
SELECT DISTINCT Patient_ID, 1 AS chd_code
FROM CodedEvent
WHERE {codes_where}
ORDER BY Patient_ID
'''
clin_df = pd.read_sql(query, cnxn, index_col='Patient_ID')
clin_df

,chd_code
Patient_ID,
84,1
201,1
228,1
301,1
401,1
...,...
1478491,1
1531920,1
1542798,1


In [8]:
codes_where = codes_to_sql_where("DMD_ID", cvd_medcodes['id'])
query = f'''
SELECT 
  med.Patient_ID,
  COUNT(med.Patient_ID) AS cvd_meds

FROM MedicationDictionary AS dict

INNER JOIN MedicationIssue AS med
ON dict.MultilexDrug_ID = med.MultilexDrug_ID

WHERE ({codes_where})

GROUP BY med.Patient_ID

ORDER BY med.Patient_ID
'''
med_df = pd.read_sql(query, cnxn, index_col='Patient_ID')
med_df.head()

,cvd_meds
Patient_ID,
2,1
18,3
28,7
34,44
41,2


In [9]:
clin_df.join(med_df, how='outer').fillna(0).head()

,chd_code,cvd_meds
Patient_ID,,
2,0.0,1.0
18,0.0,3.0
28,0.0,7.0
34,0.0,44.0
41,0.0,2.0


### Connection should be closed before restarting the kernal or closing the notebook

In [10]:
cnxn.close()